In [8]:
from langchain.chat_models import ChatOpenAI
import os

import openai

from langchain.output_parsers import PydanticOutputParser


from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.prompts import PromptTemplate

In [9]:
os.environ['OPENAI_API_KEY'] = ''
openai.api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])

In [18]:
# Define your desired data structure.

TRUE = ['yes', 'true']
FALSE = ['no', 'false']
class Binary(BaseModel):
    value: str = Field(description="yes or no value", examples=['yes', 'no'])

    def __bool__(self):
        if self.value in TRUE:
            return True
        else:
            return False

    def __str__(self):
        return self.value

binary_parser = PydanticOutputParser(pydantic_object=Binary)

class ClusterMethod(BaseModel):
    method: str = Field(description="type of clustering method used", examples=['k-mean', 'hiearchial', 'not found'])



    



template = "You are a helpful assistant in the field of {field} that search if the concept of {concept} is present or not in the text.\n {format_instructions}. \n The text is :  \n {text}\n"


neuro_concept_present_prompt = PromptTemplate(
    template=template,
    input_variables=["text", "concept"],
    partial_variables={"format_instructions": binary_parser.get_format_instructions(), "field":"neuroscience"},
)


In [27]:
is_concept_present = neuro_concept_present_prompt | llm | binary_parser
small_chain = neuro_concept_present_prompt | llm

In [26]:
a = is_concept_present.invoke({"concept":"despiking", "text":"I love despiking. It's help to understand the data"})
a

Binary(value='yes')

In [21]:
str(a)

'no'

In [25]:
neuro_concept_present_prompt.invoke({"concept":"despiking", "text":"I love life. It's help to understand the data"}).text

'You are a helpful assistant in the field of neuroscience that search if the concept of despiking is present or not in the text.\n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"value": {"title": "Value", "description": "yes or no value", "examples": ["yes", "no"], "type": "string"}}, "required": ["value"]}\n```. \n The text is :  \n I love life. It\'s help to understand the data\n'

In [29]:
small_chain.invoke({"concept":"despiking", "text":"I love despiking. It's help to understand the data"})

AIMessage(content='{"value": "yes"}')

In [34]:


class ClusterMethod(BaseModel):
    method: str = Field(description="type of clustering method used", examples=['k-mean', 'hiearchial', 'not found'])

clustering_parser = PydanticOutputParser(pydantic_object=ClusterMethod)


template = "You are a helpful assistant in the field of {field} that tell me the method used for {method}.\n {format_instructions}. \n The text is :  \n {text}\n"


clustering_prompt = PromptTemplate(
    template=template,
    input_variables=["text", "method"],
    partial_variables={"format_instructions": clustering_parser.get_format_instructions(), "field":"neuroscience"},
)

In [35]:
cluster_method_use = clustering_prompt | llm | clustering_parser

In [36]:
a = cluster_method_use.invoke({"method":"clustering", "text":"We used k-mean clustring method to analyse data."})
a

ClusterMethod(method='k-mean')